In [1]:
import sys
# for local development
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

In [2]:
%matplotlib notebook
db_connect_params = {
    "host" : "194.87.210.189",
    "port" : 59000,
    "user" : "alex",
    "password" : "Xrxcmr758",
    "database" : "rt5"
}

In [3]:
import time
import pandas as pd

In [4]:
from data_providers import DbDataProviderRT5
from data_providers import ClickHouseConnector

## Dataset

In [5]:
try:
    dataset = pd.read_csv("./test_dataset.csv", index_col='ts')
    print("Прочитано из файла")
except FileNotFoundError:
    
    ts = int(time.time())
    period = 60
    total_points = 5000

    pair = "BTC_USDT"


    #----
    with ClickHouseConnector(db_connect_params) as conn:
        data_provider = DbDataProviderRT5(conn)

        dataset = data_provider.get(
                                ts,
                                period,
                                total_points,
                                pair
                            )
    dataset.to_csv("./test_dataset.csv")
    print("Загружен новый датасет")
    
print(dataset.shape)

Прочитано из файла
(5000, 11)


In [6]:
dataset.head()

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1675190531,2023-01-31 21:42:11,23150.77,23134.951,"{'23150.77': 1.082469, '23150.78': 0.092513, '...","{'23134.95': 0.14, '23134.94': 0.033976, '2313...",0.016218,0.018967,0.035185,2,2,4
1675190591,2023-01-31 21:43:11,23136.01,23131.549,"{'23136.01': 1.082488, '23136.02': 0.050858, '...","{'23131.55': 0.033981, '23131.54': 0.43231, '2...",0.012107,0.029802,0.041909,4,5,9
1675190651,2023-01-31 21:44:11,23142.90,23138.930,"{'23142.90': 1.082488, '23142.91': 0.002142, '...","{'23138.93': 0.03397, '23138.92': 0.032715, '2...",0.025996,0.007660,0.033656,7,3,10
1675190711,2023-01-31 21:45:11,23138.96,23138.930,"{'23138.96': 1.082488, '23138.97': 0.027, '231...","{'23138.93': 0.03397, '23138.92': 0.032715, '2...",0.025015,0.047751,0.072766,6,8,14
1675190771,2023-01-31 21:46:11,23154.03,23149.602,"{'23154.03': 0.092126, '23160.37': 0.002124, '...","{'23149.60': 0.14, '23149.59': 0.002101, '2314...",0.017419,0.017783,0.035202,4,3,7


## DataPoint

In [120]:
"""
Класс DataPoint реализован, чтобы передать текущее представление точки данных в z_core.

Данный класс решает несколько проблем:
 1. На этапе обучение позвозяет передать дополнительные данные о точках в будущем
 для расчета индикатора тренда. В целевом решении эта задача будет решаться с помощью дополнительной модели.

 2.Содержит в себе необходимые методы для получения нужных представлений данных.

Основные тезисы:
 - Индексы реальные, не обнуляются (не начинаются с нуля).
 - Текущее значение имеет индекс с максимальным значением, остальные значения считаются историчсекими.
 - Информация о будущих точках представлена в отдельном поле и не пересекается с датасетом.
 - Реализация DataPoint отвязана от названий полей, чтобы была возможность работать с разными датасетами.

"""
import numpy as np

from basic_application import with_exception


class DataPointError(Exception):
    pass


class DataPoint:
    def __init__(self, data, future_points=0, observation_len=5):
        self.data = data
        self.offset = future_points
        self.observation_len = observation_len

        # Верхушка observation = текущая точка данных
        self.cursor = data.shape[0] - self.offset - 1
        self.current_idx = data.index.values[self.cursor]

        self.period = data.index[1] - data.index[0]

    def get_points(self, scale_factor=1, num=None):
        """Возвращает индексы по реперным точкам - т.е. будет соответствовать количеству запрошенных точек"""
        if num is None:
            num = self.observation_len

        if num >= 0:
            up_bound = self.cursor + 1
            low_bound = up_bound - (num - 1) * scale_factor - 1
        else:
            low_bound = self.cursor + scale_factor
            up_bound = low_bound - num * scale_factor

        idxs = self.data.index.values[low_bound : up_bound : scale_factor]
        return idxs

    def get_indexes(self, scale_factor=1, num=None):
        """Возвращает все индексы, в том числе расположенные между реперными точками"""
        if num is None:
            num = self.observation_len

        if num >= 0:
            up_bound = self.cursor + 1
            low_bound = up_bound - num * scale_factor

        else:
            low_bound = self.cursor + 1
            up_bound = low_bound - num * scale_factor

        #idxs = self.data.index.values[low_bound : up_bound : 1]
        #return idxs
        return low_bound, up_bound

    def get_values(self, name, scale_factor=1, num=None):
        if num is None:
            num = self.observation_len
            
        low_bound, up_bound = self.get_indexes(scale_factor=scale_factor, num=num)
        col = self.data.loc[:, name].values
        result = col[low_bound: up_bound]

        return result
    
    def get_value(self, name, scale_factor, idx=None):
        """Метод возвращает одно значение. По умолчанию для текущего индекса.
        Индекс можно задать из вне. Данные возвращаются с учетом scale_factor
        Расчет по одной точке (_build_point в AbstractFeatureWithHistory) для scalefactor>1 будет неточным -
        там надо считать все сразу. Надо исследовать, сходу сложно оценить.
        """
        if idx is None:
            idx = self.current_idx
            
        cursor = np.where(self.data.index.values == idx)[0][0]
        col = self.data.loc[:, name].values
        
        result = col[cursor - scale_factor + 1 : cursor+1]
        return result
        

    def get_current_index(self):
        return self.current_idx




In [126]:
current_rate = dp.get_value("highest_bid", scale_factor=3)
current_rate

array([23166.451, 23157.959, 23170.34 ])

In [106]:
col = dp.data.loc[:, "lowest_ask"].values
col

array([23138.96 , 23154.03 , 23154.03 , 23162.89 , 23163.94 , 23173.87 ,
       23182.19 , 23175.229, 23162.129, 23175.229, 23175.229, 23174.55 ])

In [107]:
cursor = 5
scale_factor = 2
result = col[cursor - scale_factor + 1 : cursor + 1]
result

array([23163.94, 23173.87])

## Datapoint Factory

In [108]:
"""
Рефакторинг DataPointFactory
Цель - запилить фабрику, работающую с любым шагом датасета

"""

#from .data_point import DataPoint
import numpy as np
import logging
from basic_application import with_exception

logger = logging.getLogger(__name__)


class DataPointFactoryError(Exception):
    pass


class DataPointFactory2:
    def __init__(self, dataset, step_size=1, offset=10, observation_len=10, future_points=0):
        """
        Все параметры указываются в единицах, без привязки к шагу датасета
        :param dataset: pandas dataframe с данными
        :param step_size: шаг по датасету. Указывается в единицах
        :param offest: 'хвост' с историческими данными - количество точек, которые надо захватить. 
        Рассчитывается как произведение n_observation на max scale_factor
        :param observation_len: количество точек в наблюдении
        :param future_points: количество точек в будущем
        """
        self.dataset = dataset
        self.offset = offset
        self.step_size = step_size
        self.observation_len = observation_len
        self.future_points = future_points

        #self.period = self.dataset.index[1] - self.dataset.index[0]
        self.max_step = self.dataset.shape[0]
        self.cursor = self.offset
        self.done = True

        self.reset()

    def reset(self):
        """Сброс в начальное состояние"""
        self.done = False

        # Курсор устанавливается на значение, которое отстоит от начала так, чтобы от начала
        # до курсора был размер данных для одного датапоинта.
        self.cursor = self.offset #min(self.dataset.index) + self.period * (self.offset - 1)
        data_point = self.get_current_step()
        return data_point
    
    def get_current_step(self):
        """ Возвращает текущий data_point"""
        data = self.dataset.iloc[self.cursor - self.offset : self.cursor + self.future_points, :]

        data_point = DataPoint(
            data,
            future_points=self.future_points,
            observation_len=self.observation_len
        )
        return data_point

    def get_next_step(self):
        """Переводит курсор на величину шага и возвращает data_point"""
        if not self.done:
            self.cursor = self.cursor + self.step_size

        if self.cursor >= self.max_step:
            self.done = True
        else:
            self.done = False

        data_point = self.get_current_step()
        return data_point, self.done


In [109]:
dpf = DataPointFactory2(dataset, step_size=3, offset=10, observation_len=10, future_points=2)

In [110]:
dp = dpf.get_current_step()

In [111]:
dp, done = dpf.get_next_step()

In [112]:
dp.data.head()

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1675190711,2023-01-31 21:45:11,23138.96,23138.930,"{'23138.96': 1.082488, '23138.97': 0.027, '231...","{'23138.93': 0.03397, '23138.92': 0.032715, '2...",0.025015,0.047751,0.072766,6,8,14
1675190771,2023-01-31 21:46:11,23154.03,23149.602,"{'23154.03': 0.092126, '23160.37': 0.002124, '...","{'23149.60': 0.14, '23149.59': 0.002101, '2314...",0.017419,0.017783,0.035202,4,3,7
1675190831,2023-01-31 21:47:11,23154.03,23152.320,"{'23154.03': 0.090048, '23162.89': 0.027313, '...","{'23152.32': 0.14, '23152.31': 0.002164, '2314...",0.113976,0.031245,0.145221,8,9,17
1675190891,2023-01-31 21:48:11,23162.89,23154.871,"{'23162.89': 0.027313, '23167.19': 0.002142, '...","{'23154.87': 0.14, '23154.86': 0.039723, '2315...",0.007418,0.018911,0.026329,2,3,5
1675190951,2023-01-31 21:49:11,23163.94,23154.850,"{'23163.94': 0.054471, '23168.30': 0.164, '231...","{'23154.85': 0.14, '23154.84': 0.431875, '2315...",0.027289,0.015072,0.042361,11,2,13


In [113]:
import numpy as np

In [114]:
result = np.where(dp.data.index.values == 1675191071)

In [115]:
result[0][0]

6

## Perfomance test

In [116]:
import cProfile

In [117]:

def perfomance_test(dataset):
    dpf = DataPointFactory2(dataset, step_size=1, offset=100, observation_len=10, future_points=0)
    dp = dpf.reset()
    done = False
    scale_factor = (1,2,5,10)

    while not done:
        dp, done = dpf.get_next_step()
        for sf in scale_factor:
            dp.get_points(scale_factor=sf)
            dp.get_indexes(scale_factor=sf)
            dp.get_values("lowest_ask", scale_factor=sf)
            dp.get_value("lowest_ask", scale_factor=sf)


In [118]:
#%prun perfomance_test(dataset)    

In [119]:
cProfile.run("perfomance_test(dataset)")

         5743130 function calls (5733324 primitive calls) in 2.943 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.060    0.060    2.943    2.943 3631435050.py:1(perfomance_test)
        1    0.000    0.000    0.000    0.000 3878179567.py:20(__init__)
        2    0.000    0.000    0.000    0.000 3878179567.py:43(reset)
     4902    0.015    0.000    0.462    0.000 3878179567.py:53(get_current_step)
     4900    0.007    0.000    0.468    0.000 3878179567.py:64(get_next_step)
     4902    0.017    0.000    0.048    0.000 486743214.py:27(__init__)
    19600    0.030    0.000    0.034    0.000 486743214.py:38(get_points)
    39200    0.014    0.000    0.014    0.000 486743214.py:53(get_indexes)
    19600    0.048    0.000    1.190    0.000 486743214.py:70(get_values)
    19600    0.162    0.000    1.183    0.000 486743214.py:92(get_value)
    19600    0.018    0.000    0.049    0.000 <__array_function__ interna

In [72]:
cProfile.run("perfomance_test(dataset)")

         7786430 function calls (7757024 primitive calls) in 4.132 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4902    0.017    0.000    0.047    0.000 2861719652.py:27(__init__)
    19600    0.040    0.000    0.043    0.000 2861719652.py:38(get_points)
    39200    0.014    0.000    0.014    0.000 2861719652.py:53(get_indexes)
    19600    0.050    0.000    1.228    0.000 2861719652.py:70(get_values)
    19600    0.047    0.000    2.306    0.000 2861719652.py:80(get_value)
        1    0.088    0.088    4.132    4.132 3631435050.py:1(perfomance_test)
        1    0.000    0.000    0.000    0.000 3878179567.py:20(__init__)
        2    0.000    0.000    0.001    0.000 3878179567.py:43(reset)
     4902    0.015    0.000    0.452    0.000 3878179567.py:53(get_current_step)
     4900    0.007    0.000    0.458    0.000 3878179567.py:64(get_next_step)
        1    0.000    0.000    4.132    4.132 <string>:1(<module>)
 

In [510]:
dataset.iloc[:30,:].index

Int64Index([1675275006, 1675275066, 1675275126, 1675275186, 1675275246,
            1675275306, 1675275366, 1675275426, 1675275486, 1675275546,
            1675275606, 1675275666, 1675275726, 1675275786, 1675275846,
            1675275906, 1675275966, 1675276026, 1675276086, 1675276146,
            1675276206, 1675276266, 1675276326, 1675276386, 1675276446,
            1675276506, 1675276566, 1675276626, 1675276686, 1675276746],
           dtype='int64', name='ts')

In [501]:
dp = DataPoint(dataset.iloc[:30,:], future_points=10)

In [502]:
dp.cursor

19

In [503]:
dp.current_idx

1675276146

In [504]:
dp.get_points(scale_factor=1, num=-3)

array([1675276206, 1675276266, 1675276326])

In [508]:
dp.get_indexes(scale_factor=3, num=-3)

array([1675276206, 1675276266, 1675276326, 1675276386, 1675276446,
       1675276506, 1675276566, 1675276626, 1675276686])

In [506]:
dp.get_values('lowest_ask', scale_factor=2)

array([22996.69 , 22990.51 , 22997.148, 22990.361, 22984.67 , 22995.42 ,
       22995.42 , 22995.12 , 23011.629, 23036.31 ], dtype=float32)

In [475]:
dp.get_values('lowest_ask', scale_factor=4)

array([22996.69 , 22990.51 , 22997.148, 22990.361, 22984.67 , 22995.42 ,
       22995.42 , 22995.12 , 23011.629, 23036.31 , 23105.99 , 23100.34 ,
       23074.36 , 23076.55 , 23070.32 , 23077.92 , 23068.43 , 23075.94 ,
       23057.29 , 23056.47 ], dtype=float32)

In [515]:
s = dp.get_value('lowest_ask', scale_factor=1)
s.values[0]

23036.31

In [448]:
dataset.iloc[:30,:].tail(10)

,dt,lowest_ask,highest_bid,asks,bids,buy_vol,sell_vol,total_vol,buy_num,sell_num,total_num
ts,,,,,,,,,,,
1675276206,2023-02-01 21:30:06,23105.990234,23091.648438,"{'23105.99': 1.130567, '23106.00': 0.1638, '23...","{'23091.65': 0.14, '23091.64': 0.433057, '2309...",0.049586,0.173875,0.223461,8,12,20
1675276266,2023-02-01 21:31:06,23100.339844,23084.921875,"{'23100.34': 1.130567, '23100.35': 0.072796, '...","{'23084.92': 0.072611, '23082.05': 0.433237, '...",0.177567,0.018250,0.195817,8,9,17
1675276326,2023-02-01 21:32:06,23074.359375,23061.210938,"{'23074.36': 1.123013, '23074.37': 0.071934, '...","{'23061.21': 0.083628, '23061.20': 0.433628, '...",0.007781,0.004471,0.012252,6,2,8
1675276386,2023-02-01 21:33:06,23076.550781,23066.138672,"{'23076.55': 1.122374, '23076.56': 0.001306, '...","{'23066.14': 0.0069, '23061.21': 0.077366, '23...",0.005242,0.012612,0.017854,1,3,4
1675276446,2023-02-01 21:34:06,23070.320312,23063.740234,"{'23070.32': 1.125054, '23070.33': 0.002106, '...","{'23063.74': 0.004224, '23063.73': 0.14, '2306...",0.007578,0.007244,0.014822,3,2,5
1675276506,2023-02-01 21:35:06,23077.919922,23069.179688,"{'23077.92': 1.119862, '23077.93': 0.083979, '...","{'23069.18': 0.00027, '23069.17': 0.14, '23069...",0.008497,0.012700,0.021197,7,3,10
1675276566,2023-02-01 21:36:06,23068.429688,23060.958984,"{'23068.43': 1.11961, '23068.44': 0.00191, '23...","{'23060.96': 0.009673, '23060.95': 0.000271, '...",0.014755,0.011496,0.026251,7,6,13
1675276626,2023-02-01 21:37:06,23075.939453,23060.939453,"{'23075.94': 1.122408, '23075.95': 0.002091, '...","{'23060.94': 0.004195, '23060.93': 0.433633, '...",0.038254,0.019944,0.058198,6,5,11
1675276686,2023-02-01 21:38:06,23057.289062,23046.791016,"{'23057.29': 1.117885, '23057.30': 0.002162, '...","{'23046.79': 0.14, '23046.78': 0.008722, '2304...",0.027223,0.018791,0.046014,10,3,13


In [195]:
dataset.index.values

array([1675275006, 1675275066, 1675275126, 1675275186, 1675275246,
       1675275306, 1675275366, 1675275426, 1675275486, 1675275546,
       1675275606, 1675275666, 1675275726, 1675275786, 1675275846,
       1675275906, 1675275966, 1675276026, 1675276086, 1675276146,
       1675276206, 1675276266, 1675276326, 1675276386, 1675276446,
       1675276506, 1675276566, 1675276626, 1675276686, 1675276746,
       1675276806, 1675276866, 1675276926, 1675276986, 1675277046,
       1675277106, 1675277166, 1675277226, 1675277286, 1675277346,
       1675277406, 1675277466, 1675277526, 1675277586, 1675277646,
       1675277706, 1675277766, 1675277826, 1675277886, 1675277946,
       1675278006, 1675278066, 1675278126, 1675278186, 1675278246,
       1675278306, 1675278366, 1675278426, 1675278486, 1675278546,
       1675278606, 1675278666, 1675278726, 1675278786, 1675278846,
       1675278906, 1675278966, 1675279026, 1675279086, 1675279146,
       1675279206, 1675279266, 1675279326, 1675279386, 1675279